In [3]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

In [4]:
representations = np.load('/u/scr/ethanchi/relationOutputs/finnish/representations.npy')
morph = np.load('/u/scr/ethanchi/relationOutputs/finnish/morph.npy')
pos = np.load('/u/scr/ethanchi/relationOutputs/finnish/pos.npy')

In [5]:
CASES = ["Nom", "Acc", "Gen", "Ins", "Par", "Ess", "Tra", "Com", "Abe", "Ine", "Ill", "Ela", "Ade", "All", "Abl"]
NUM_CASES = len(CASES)

In [6]:
data = morph[pos=='NOUN']
reps = representations[pos=='NOUN']

print(data.shape)

labels = np.zeros([data.shape[0], NUM_CASES])

for idx, case in enumerate(CASES):
  is_case = np.vectorize(lambda x: x.startswith("Case=" + case))
  labels[:, idx] = is_case(data)

print(labels.shape)
  
labels = np.argmax(labels, axis=1)
print(labels.shape)
print(labels)

(5103,)
(5103, 15)
(5103,)
[ 2  0 12 ...  9 10  2]


In [7]:
label_tensor = torch.Tensor(labels)
data_tensor = torch.Tensor(reps)

indices = np.arange(label_tensor.shape[0])
np.random.shuffle(indices)

label_tensor = label_tensor[indices]
data_tensor = data_tensor[indices]

sep = int(0.9 * label_tensor.shape[0])

train_labels = label_tensor[:sep]
train_data = data_tensor[:sep]

test_labels = label_tensor[sep:]
test_data = data_tensor[sep:]

In [8]:
class CaseClassifier(nn.Module):
  def __init__(self, rep_size, out_size):
    super(CaseClassifier, self).__init__()
    self.linear = nn.Linear(rep_size, out_size)
  def forward(self, vectors):
    return nn.functional.log_softmax(self.linear(vectors), dim=1)
  
NUM_EPOCHS = 2000

model = CaseClassifier(representations.shape[-1], NUM_CASES)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = nn.CrossEntropyLoss()



for epoch in range(NUM_EPOCHS):
  optimizer.zero_grad()
  train_outputs = model(train_data)
  train_loss = loss(train_outputs, train_labels.long())
  train_loss.backward()
  optimizer.step()
  if epoch % 100 == 0:
    with torch.no_grad():
      test_outputs = model(test_data)
      test_loss = loss(test_outputs, test_labels.long())
      train_accuracy = len(np.where(torch.argmax(train_outputs, dim=1) == train_labels.long())[0]) / train_outputs.shape[0]
      test_accuracy = len(np.where(torch.argmax(test_outputs, dim=1) == test_labels.long())[0]) / test_outputs.shape[0]
      print("Epoch {}, train loss {:5.3f}, train accuracy {:2.3f}%, test loss {:5.3f}, test accuracy {:2.3f}%".format(epoch, train_loss, train_accuracy*100, test_loss, test_accuracy*100))



Epoch 0, train loss 2.768, train accuracy 4.508%, test loss 2.733, test accuracy 5.284%
Epoch 100, train loss 1.648, train accuracy 55.009%, test loss 1.690, test accuracy 52.055%
Epoch 200, train loss 1.190, train accuracy 73.454%, test loss 1.258, test accuracy 70.646%
Epoch 300, train loss 0.923, train accuracy 81.773%, test loss 1.004, test accuracy 78.669%
Epoch 400, train loss 0.751, train accuracy 86.781%, test loss 0.839, test accuracy 83.366%
Epoch 500, train loss 0.632, train accuracy 89.460%, test loss 0.726, test accuracy 85.323%
Epoch 600, train loss 0.546, train accuracy 91.115%, test loss 0.644, test accuracy 87.084%
Epoch 700, train loss 0.480, train accuracy 92.247%, test loss 0.583, test accuracy 88.454%
Epoch 800, train loss 0.429, train accuracy 93.075%, test loss 0.535, test accuracy 88.650%
Epoch 900, train loss 0.387, train accuracy 93.706%, test loss 0.498, test accuracy 89.237%
Epoch 1000, train loss 0.353, train accuracy 94.207%, test loss 0.468, test accuracy

In [9]:
en_representations = np.load('/u/scr/ethanchi/relationOutputs/english/representations.npy')
en_pos = np.load('/u/scr/ethanchi/relationOutputs/english/pos.npy')
en_extra = np.load('/u/scr/ethanchi/relationOutputs/english/data.npy')

en_extra = en_extra[en_pos=='NOUN']
en_representations = en_representations[en_pos=='NOUN']

print(en_representations.shape)

(4195, 768)


In [10]:
en_inputs = torch.Tensor(en_representations)
with torch.no_grad():
  en_outputs = model(en_inputs)
  en_best_labels = torch.argmax(en_outputs, dim=1)
  
for case, num in zip(CASES, torch.bincount(en_best_labels.int())):
  print(f"{case}\t{num}")
  
  


Nom	3239
Acc	0
Gen	293
Ins	2
Par	466
Ess	50
Tra	4
Com	0
Abe	0
Ine	64
Ill	12
Ela	11
Ade	40
All	9
Abl	5


In [11]:
for idx, (case, num) in enumerate(zip(CASES, torch.bincount(en_best_labels.int()))):
  print(f"{case}\t{num}")
  for (sentence, sent_idx, _) in en_extra[en_best_labels.numpy()==idx][:10]:
      sent_idx = int(sent_idx)
      words = sentence.split(' ')
      words[sent_idx] = "**" + words[sent_idx] + "**"
      printmd(" ".join(words))

Nom	3239


From the AP comes this **story** :

President Bush on Tuesday nominated two **individuals** to replace retiring jurists on federal courts in the Washington area .

President Bush on Tuesday nominated two individuals to replace retiring **jurists** on federal courts in the Washington area .

President Bush on Tuesday nominated two individuals to replace retiring jurists on federal courts in the Washington **area** .

...

It 's on loan , by the **way** , from a guy named Joe O'Neill in Midland , Texas .

Raw data is on the fisrt tab of the file to the **right** .

Normally this author is semi-objective ( what blogger is ) but on the Seattlepi.com ( see source link ) Palestinian Leader Mahmoud Abbas has demanded that Israel leave all the land that it occupies before the 1967 border ( which includes East Jerusalem by the **way** ) .

" We 're not happy with Hamas 's position at this **stage** and in light of the Israeli escalation against the Palestinians , " commented Khader Habib , a senior Islamic Jihad official .

Miramar California is a **bit** north of San Diego .

In [12]:
en_non_nom_outputs = en_outputs[:, 1:]
en_rev_best_labels = torch.argmax(en_non_nom_outputs, dim=1)
  
for case, num in zip(CASES[1:], torch.bincount(en_rev_best_labels.int())):
  print(f"{case}\t{num}")
  
  

Acc	0
Gen	1520
Ins	14
Par	1951
Ess	112
Tra	12
Com	6
Abe	1
Ine	164
Ill	117
Ela	31
Ade	205
All	46
Abl	16


In [13]:
for idx, (case, num) in enumerate(zip(CASES[1:], torch.bincount(en_rev_best_labels.int()))):
  print(f"{case}\t{num}")
  print(en_extra[en_rev_best_labels.numpy()==idx][:10])

Acc	0
[]
Gen	1520
[['President Bush on Tuesday nominated two individuals to replace retiring jurists on federal courts in the Washington area .'
  '6' 'individuals']
 ['The sheikh in wheel - chair has been attacked with a F - 16 - launched bomb .'
  '1' 'sheikh']
 ['The sheikh in wheel - chair has been attacked with a F - 16 - launched bomb .'
  '11' 'F']
 ["Today 's incident proves that Sharon has lost his patience and his hope in peace ."
  '2' 'incident']
 ["Today 's incident proves that Sharon has lost his patience and his hope in peace ."
  '9' 'patience']
 ['In the eastern city of Baqubah , guerrillas detonated a car bomb outside a police station , killing several people .'
  '10' 'car']
 ['In the eastern city of Baqubah , guerrillas detonated a car bomb outside a police station , killing several people .'
  '14' 'police']
 ['The US lost yet another helicopter to hostile fire near Habbaniyah in the Sunni heartland , but this time the crew was safe .'
  '20' 'crew']
 ['( I hope th

In [14]:
def printmd(string):
    display(Markdown(string))
for idx, (case, num) in enumerate(zip(CASES[1:], torch.bincount(en_rev_best_labels.int()))):
  print(f"{case}\t{num}")
  for (sentence, sent_idx, _) in en_extra[en_rev_best_labels.numpy()==idx][:10]:
    sent_idx = int(sent_idx)
    words = sentence.split(' ')
    words[sent_idx] = "**" + words[sent_idx] + "**"
    printmd(" ".join(words))

Acc	0
Gen	1520


President Bush on Tuesday nominated two **individuals** to replace retiring jurists on federal courts in the Washington area .

The **sheikh** in wheel - chair has been attacked with a F - 16 - launched bomb .

The sheikh in wheel - chair has been attacked with a **F** - 16 - launched bomb .

Today 's **incident** proves that Sharon has lost his patience and his hope in peace .

...

Vangie McGilloway Constellation Power Source , Inc. ( " CPS " ) 111 Market Place **Ste** 500 Baltimore , MD 21202 Phone 410-468-3798 Fax 410-468-3499 Email vangie.mcgilloway@powersrc.com

Sungjoo Lee Stochastic Processes and Simulation in Finance Georgia Institute of Technology School of Industrial and Systems Engineering E-mail **address** : goldconn@isye.gatech.edu gte114t@prism.gatech.edu Home : 404-449-1026 Office : Coming Soon !

I am expecting to pay something in the $ 3, to $ 5,000 **range** .

In Indian national chart besides the transit affliction to transit Mercury by Rahu and Venus the natal Saturn and Venus are also under the transit affliction of Rahu which has resulted into floods in many parts of the country taking **toll** of human lives , properties , etc .

Normally this author is semi-objective ( what blogger is ) but on the Seattlepi.com ( see source link ) Palestinian Leader Mahmoud Abbas has demanded that Israel leave all the land that it occupies before the 1967 border ( which includes East Jerusalem by the **way** ) .

In [35]:
for idx, (case, num) in enumerate(zip(CASES, torch.bincount(en_best_labels.int()))):
  print(f"{case}\t{num}")
#   if num == 0: continue
  for (sentence, sent_idx, _), score in zip(en_extra[np.argsort(-en_outputs[:, idx])][:10], en_outputs[np.argsort(-en_outputs[:, idx]), idx][:10]):
      sent_idx = int(sent_idx)
      words = sentence.split(' ')
      words[sent_idx] = "**" + words[sent_idx] + "**"
      words = " ".join(words)
      words = f"`({np.exp(score):5.3f})` " + words
      printmd(words)

Nom	3239


`(0.999)` My old **bed** got tossed in a dumpster .

`(0.998)` The **owner** Karla is welcoming and fun .

`(0.998)` The chicken cordon - **blu** was tasty and came in a huge portion size for the money .

`(0.998)` But **iPhone** was the first " Officiol " touch screen smart phone .

...

`(0.277)` But if -- there 's a painting on the **wall** in the Oval Office that shows a horseman charging up a steep cliff , and there are at least two other horsemen following .

`(0.268)` The US National Hurricane Center said Rita was " extremely dangerous " and could become a top - level category five storm on the Saffir - Simpson **scale** .

`(0.260)` Normally this author is semi-objective ( what blogger is ) but on the Seattlepi.com ( see source link ) Palestinian Leader Mahmoud Abbas has demanded that Israel leave all the land that it occupies before the 1967 border ( which includes East Jerusalem by the **way** ) .

`(0.243)` In addition , I received feedback from our Gas Desk that the access to the Gas **segment** of Enron On - Line was cut off to CPS --- do you know who would handle this at Enron that we can speak to ?

`(0.228)` It 's on loan , by the **way** , from a guy named Joe O'Neill in Midland , Texas .